In [76]:
import pandas as pd
import matplotlib.pyplot as plt
from urllib.request import urlopen  
import os.path as osp
import os
import logging
import gzip
import shutil
from glob import glob
logging.getLogger().setLevel('INFO')

In [77]:
def download_file(url_str, path):
    url = urlopen(url_str)
    output = open(path, 'wb')
    output.write(url.read())
    output.close()

In [78]:
BASE_URL = 'https://archive.ics.uci.edu/ml/machine-learning-databases/'
DATA_DIR = 'covtype'

DATA_URL = BASE_URL + DATA_DIR + "/covtype.data.gz"

os.makedirs(osp.join(DATA_DIR, 'archives'), exist_ok=True)

logging.info("downloading & extracting file %s", DATA_URL)
archive_path = osp.join(DATA_DIR, 'archives', osp.basename(DATA_URL))
download_file(DATA_URL, archive_path)

features_names = {'Cover_Type':[54],'Elevation':[0],
                  'Aspect':[1],'Slope':[2],
                  'Horizontal_Distance_To_Hydrology':[3],
                  'Vertical_Distance_To_Hydrology':[4],
                  'Horizontal_Distance_To_Roadways':[5],
                  'Hillshade_9am':[6],'Hillshade_Noon':[7],
                  'Hillshade_3pm':[8],
                  'Horiontal_Distance_To_Fire_Points':[9],
                  'Wilderness_Area':[10,11,12,13],
                  'Soil_Type':list(range(14,54))}

df_covtype = pd.read_csv(archive_path,header=None,
                         parse_dates=features_names,compression='gzip')
df_covtype.convert_objects(convert_numeric=True)

INFO:root:downloading & extracting file https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz
C:\Anaconda3\envs\python36\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


,Cover_Type,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horiontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type
0,5,2596,51,3,258,0,510,221,232,148,6279,1 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1,5,2590,56,2,212,-6,390,220,235,151,6225,1 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2,2,2804,139,9,268,65,3180,234,238,135,6121,1 0 0 0,0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 ...
3,2,2785,155,18,242,118,3090,238,238,122,6211,1 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4,5,2595,45,2,153,-1,391,220,234,150,6172,1 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
5,2,2579,132,6,300,-15,67,230,237,140,6031,1 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
6,5,2606,45,7,270,5,633,222,225,138,6256,1 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
7,5,2605,49,4,234,7,573,222,230,144,6228,1 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
8,5,2617,45,9,240,56,666,223,221,133,6244,1 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
9,5,2612,59,10,247,11,636,228,219,124,6230,1 0 0 0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...


In [80]:
type(df_covtype.Elevation[0])

str

In [81]:
df_covtype['Elevation'] = [x.astype(float) for x in df_covtype['Elevation']]

AttributeError: 'str' object has no attribute 'astype'

In [22]:
pd.plotting.scatter_matrix(covtype[['Elevation','Aspect','Slope']])

ValueError: num must be 1 <= num <= 0, not 1

In [54]:
type(range(5))

range